In [1]:
#Sentiment Analysis on IMDB Reviews using LSTM
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
english_stops = set(stopwords.words('english'))   # remove uncessary word

In [4]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\dell\AppData\Local\Temp\ipykernel_8372\4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
17331    [why, released, i, kind, thought, must, bit, r...
13388    [i, really, shocked, see, number, reviews, lam...
11400    [ok, if, looking, another, bastketball, diarie...
45159    [one, best, lackawanna, bluesgreat, movie, gre...
39224    [not, stunner, good, movie, see, twice, bill, ...
                               ...                        
13096    [i, true, fan, original, cracker, series, dvd,...
697      [finally, released, good, modesty, blaise, mov...
9314     [curse, monkey, island, treasure, opinion, ser...
8550     [what, one, say, wilder, film, human, real, st...
17714    [a, pretty, worthless, made, television, movie...
Name: review, Length: 40000, dtype: object 

45471    [through, hour, running, length, crash, charts...
49054    [feeding, masses, one, many, recent, mediocre,...
35621    [well, first, big, fanatic, horror, movies, ho...
40273    [how, movie, described, oh, yeah, i, got, wret...
31232    [i, originally, saw, dark, comedy, around, cab...
 

In [6]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [7]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 365  530    1 ...    0    0    0]
 [   1   14 2125 ...  732 4462   40]
 [ 489   55  164 ...    0    0    0]
 ...
 [2531 3055  864 ...  720  987 1154]
 [ 106    5   56 ...    0    0    0]
 [  39   89 3791 ...    0    0    0]] 

Encoded X Test
 [[3905  432  551 ...   32   76  492]
 [6577 4742    5 ...    0    0    0]
 [  16   23   98 ...    0    0    0]
 ...
 [2236   39  562 ...    0    0    0]
 [ 219  437   34 ...    0    0    0]
 [  14   18  365 ...    0    0    0]] 

Maximum review length:  130


In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()

# Make sure x_train and x_test are lists of strings
x_train = x_train.tolist()
x_test = x_test.tolist()

token.fit_on_texts(x_train)

x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)


Encoded X Train
 [[ 378  478    2 ...    1    1    1]
 [   2   15 1933 ...  750 4464   34]
 [ 435   55  168 ...    1    1    1]
 ...
 [2569 3085  815 ...  648 1070 1191]
 [ 102    6   56 ...    1    1    1]
 [  36   89 3746 ...    1    1    1]] 

Encoded X Test
 [[3798  412  573 ...   43   75  470]
 [6918 4843    6 ...    1    1    1]
 [  17   21   96 ...    1    1    1]
 ...
 [2233   36  587 ...    1    1    0]
 [ 199  399   46 ...    1    1    1]
 [  15   18  378 ...    1    1    1]] 

Maximum review length:  130


In [9]:
# ARCHITECTURE       #long short term model   rnn
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()   #for stacking
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))             #layers
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

c:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
checkpoint = ModelCheckpoint(       #confirm
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [11]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5409 - loss: 0.6798
Epoch 1: accuracy improved from None to 0.57313, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 87ms/step - accuracy: 0.5731 - loss: 0.6659
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.6052 - loss: 0.6465
Epoch 2: accuracy improved from 0.57313 to 0.62268, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - accuracy: 0.6227 - loss: 0.6371
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6197 - loss: 0.6386
Epoch 3: accuracy did not improve from 0.62268
313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - accuracy: 0.5909 - loss: 0.6563
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.6048 - loss: 0.6486
Epoch 4: accuracy did not improve from 0.62268
313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - accuracy: 0.6025 - loss: 0.6470
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6737 - loss: 0.5791
Epoch 5: accuracy improved from 0.62268 to 0.66647, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 95ms/step - accuracy: 0.6665 - loss: 0.5893


In [12]:
# Get prediction probabilities
y_pred_prob = model.predict(x_test, batch_size=128)

# Convert probabilities to class labels (0 or 1)
y_pred = (y_pred_prob > 0.5).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format(true/len(y_pred)*100))


79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
Correct Prediction: 5486
Wrong Prediction: 4514
Accuracy: 54.86%


In [13]:
loaded_model = load_model('models/LSTM.h5')

In [21]:
review = str(input('Movie Review: '))

In [15]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [16]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [17]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
[[0.4707707]]


In [20]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
